## Project Description

## Import libraries

In [1]:
import pandas as pd
import clickhouse_connect
import os
import missingno as msno
from datetime import datetime

from dotenv import load_dotenv
load_dotenv()

# constants
CH_USER = os.getenv("CH_USER")
CH_PASS = os.getenv("CH_PASS")
CH_IP = os.getenv('CH_IP')

# from tools import create_db_table_from_df, pd_tools, spark_tools

root_path = "."
tmp_path = f'{root_path}/tmp'
data_path = f'{root_path}/data'
data_test_path = f'{data_path}/test_data'
data_train_path = f'{data_path}/train_data'
prod_data_path = f'{data_path}/prod_data'
new_data_path=f'{data_path}/new_data'
tmp_data_path=f'{data_path}/tmp_data'

ch_client = clickhouse_connect.get_client(host=CH_IP, port=8123, username=CH_USER, password=CH_PASS)

## Data load

# Calculate metric

Let's describe final metric. As a first step, all predicted triples $(x,y,yaw)$ are being converted into 2 points $[(x_1, y_1), (x_2, y_2)]$ in the following way:
$$
(x_1, y_1) = (x, y), \\
(x_2, y_2) = (x_1, y_1) + S \times (yaw_x, yaw_y)
$$  

where $S = 1$. In other words, we build a directed segment of length $1$. These points then used in the metric calculation.


Metric for a single pose (rmse):

$$
pose\_metric = \sqrt{ \frac{\displaystyle\sum_{j=1}^{k} {(x_j-\hat{x_j})^2 + (y_j-\hat{y_j})^2}}{k} }
$$

where $k$ - number of points that describe single pose (in our case $k=2$).

Metric for a testcase:

$$
testcase\_metric = \frac{1}{n}  \displaystyle\sum_{i=1}^{n}pose\_metric_i
$$

where $n$ - number of localization points to predict.

And, final metric for a whole dataset:

$$
dataset\_metric = \frac{1}{n}  \displaystyle\sum_{i=1}^{n}testcase\_metric_i
$$

where $n$ - number of test cases.
